## Utilities for Exam

In [1]:
import numpy as np
import sympy as sim
import scipy as sp
import math
import matplotlib.pyplot as plt
import SolveTriangular as st

### Matrix Conditions
- The following blocks will display some function to check matrixes constraints.

In [2]:
# Check if a matrix is positively defined
def is_positively_defined(A):
    return np.all(np.linalg.eigvals(A) > 0)

# Check if a matri is symmetric
def is_symmetric(A, rtol=1e-05, atol=1e-08):
    return np.allclose(A, A.T, rtol=rtol, atol=atol)

# Check if the matrix has max rank
def has_max_rank(A):
    return A.shape[0]==np.linalg.matrix_rank(A)

# Check if its a square matrix
def is_square(A):
    return A.shape[0]==A.shape[1]

# Check if the matrix is well or ill conditioned
"""
Returns:
- 1 if the matrix is well conditioned
- 0 if is averagely poorly ill
- -1 if is ill conditioned
"""
def matrix_condition(A):
    cond=np.linalg.cond(A)
    if cond < 10**2:
        return 1
    elif cond>=10**2 and cond<=10**3:
        return 0
    else:
        return -1

# Check if the singularity of a matrix
def is_not_singular(A):
    return np.linalg.det(A)!=0
    
# Check if the matrix is dense
def is_dense(A):
    return np.count_nonzero(A)>(1/3)*A.shape[0]*A.shape[1]

# Check if the matrix is strictly diagonally dominant
def is_diagonally_dominant(A):
    abs_A = np.abs(A)
    return np.all(2*np.diag(abs_A) > np.sum(abs_A, axis=1)) #|Aii| ≥ ∑j≠i |Aij|  =>  2|Aii| ≥ ∑j |Aij|.


## Find Zeroes of a Function
---
> Methods:

1. Bisection.
2. Regula Falsi
3. Metodo delle Corde
4. Metodo delle Secanti
5. Newton, Newton Modified

In [3]:
def order_valuation(xk,iterazioni):
     #Vedi dispensa allegata per la spiegazione

    k=iterazioni-4
    p=np.log(abs(xk[k+2]-xk[k+3])/abs(xk[k+1]-xk[k+2]))/np.log(abs(xk[k+1]-xk[k+2])/abs(xk[k]-xk[k+1]));
    
    ordine=p
    return ordine


### Bisection

In [4]:
def bisection(f, a, b, tolx):
    fa=f(a)
    fb=f(b)
    
    if fa*fb >= 0:
        print("Failed to apply method")
        return None, None, None

    it = 0
    v_xk = []

    while abs(b-a)> tolx:
        xk = a+(b-a)/2 # key operation
        v_xk.append(xk)
        it+=1
        fxk=f(xk)
        if fxk==0:
            return xk, it, v_xk
        
        if fa*fxk < 0: # la radice si trova nell'intervallo [a, xk]
            b = xk
            fb=fxk
        elif fxk*fb < 0: # intervallo [xk, b]
            a=xk
            fa=fxk
    return xk, it, v_xk

### Regula Falsi

In [5]:
def falses(f, a, b, tolx, tolf, maxit):
    fa=f(a)
    fb=f(b)

    if fa*fb >= 0:
        print("Failed to apply method")
        return None, None, None
    
    it = 0
    v_xk=[]
    fxk=100
    error=100 # difference between 2 consecutive calculation
    xprec=a
    
    while it<maxit and abs(fxk)> tolf and error > tolx:
        xk = a-fa*(b-a)/(fb-fa) # key operation
        v_xk.append(xk)
        it+=1
        fxk=f(xk)
        if fxk==0:
            return xk, it, v_xk

        if fa*fxk < 0: # la radice si trova nell'intervallo [a, xk]
            b = xk
            fb=fxk
        elif fxk*fb < 0: # intervallo [xk, b]
            a=xk
            fa=fxk
        if xk!=0:
            error=abs(xk-xprec)/abs(xk)
        else:
            error=abs(xk-xprec)
        xprec=xk
    return xk, it, v_xk

### Metodo delle Corde

In [6]:
def corde(f, x0, coeff_ang, tolx, tolf, nmax):
    xk=[] # iterati successivi
    it=0
    errorex=1+tolx # Inizializziamo gli errori in modo che entri nel while almeno una volta
    erroref=1+tolf #

    while it<nmax and erroref>=tolf and errorex>=tolx :
        fx0=f(x0)       #|
        d=fx0/coeff_ang #| Key Operations
        x1=x0-d         #|
        fx1=f(x1)
        erroref=np.abs(fx1)
        if x1!=0:
            errorex=abs(d)/abs(x1)
        else:
            errorex=abs(d)
        x0=x1
        it=it+1
        xk.append(x1)
    if(it==nmax):
        print('Max Iteration Reached')
    return x1,it,xk

### Metodo delle Secanti

In [7]:
def secanti(f, x0, x1, tolx, tolf, nmax):
    xk=[] # iterati successivi
    it=0
    errorex=1+tolx # Inizializziamo gli errori in modo che entri nel while almeno una volta
    erroref=1+tolf #

    while it<nmax and erroref>=tolf and errorex>=tolx :
        fx0=f(x0)                 #
        fx1=f(x1)                 # Key Operations
        d=fx1*((x1-x0)/(fx1-fx0)) #
        
        x1new=x1-d 
        
        fx1=f(x1new)
        
        xk.append(x1new)
        
        if x1new!=0:
            errorex=abs(d)/abs(x1new)
        else:
            errorex=abs(d)
            
        erroref=abs(fx1)
        
        x0=x1
        x1=x1new
        it=it+1
    if(it==nmax):
        print('Max Iteration Reached')
    return x1,it,xk

### Newton

In [8]:
def newton(f,d_f,x0,tolx,tolf,nmax):
  
    xk=[]
    
    it=0
    errorx=1+tolx
    errorf=1+tolf
    
    while it<nmax and errorf>=tolf and errorx >= tolx:
        fx0=f(x0)
        if abs(d_f(x0))<=np.spacing(1):
            print("First Derivate = 0 in x0")
            return None, None, None
        
        d=f_x0/d_f(x0)
        x1=x0-d # Key operations
        
        fx1=f(x1)
        errorf=np.abs(fx1)
        if x1!=0:
            errorx=abs(d)/abs(x1)
        else:
            errorx=abs(d)/abs(x1) 
        
        it=it+1
        x0=x1
        xk.append(x1)
      
    if it==nmax:
        print('Max Iteration Reached')
        
    
    return x1,it,xk

def newton_molteplicity(f,d_f,m,x0,tolx,tolf,nmax):    
    xk=[]
    
    it=0
    errorx=1+tolx
    errorf=1+tolf
    
    while it<nmax and errorf>=tolf and errorx >= tolx:
        fx0=f(x0)
        if abs(d_f(x0))<=np.spacing(1):
            print("First Derivate = 0 in x0")
            return None, None, None
        
        d=f_x0/d_f(x0)
        x1=x0-m*d # Key operations
        
        fx1=f(x1)
        errorf=np.abs(fx1)
        if x1!=0:
            errorx=abs(d)/abs(x1)
        else:
            errorx=abs(d)/abs(x1) 
        
        it=it+1
        x0=x1
        xk.append(x1)
      
    if it==nmax:
        print('Max Iteration Reached')  
    return x1,it,xk

## Systems
---

### Non-Linear System Solution

#### Newton Rapshon

In [9]:
def newton_rapshon(init_guess,f,J,tolx,tolf,nmax):
    X=np.array(init_guess, dtype=float)
    
    it=0
    errorf=1+tolf
    errorx=1+tolx
    error=[]

    while it<nmax and errorf>=tolf and errorx>=tolx:
        jx=J(X[0],X[1])
        
        if np.linalg.det(jx)==0:
            print("J-Matrix does not have max rank")
            return None,None,None
        
        fx=np.array(f(X[0],X[1]))
        fx=fx.squeeze()
        
        s=np.linalg.solve(jx,-fx)

        Xnew=X+s
        XnNorm=np.linalg.norm(Xnew,1)

        if XnNorm!=0:
            errorx=np.linalg.norm(s,1)/XnNorm
        else:
            errorx=np.linalg.norm(s,1)

        error.append(errorx)
        fnew=f(Xnew[0],Xnew[1])
        errorf=np.linalg.norm(fnew.squeeze(),1)

        X=Xnew
        it=it+1
    return X,it,error

#### Newton Rapshon Corde

In [10]:
def newton_rapshon_corde(init_guess,f,J,tolx,tolf,nmax):
    X=np.array(init_guess)
    
    it=0
    errorf=1+tolf
    errorx=1+tolx
    error=[]
    
    jx=J(X[0],X[1]) # calculate jx only one time
    
    if np.linalg.det(jx)==0:
        print("J-Matrix does not have max rank")
        return None,None,None

    while it<nmax and errorf>=tolf and errorx>=tolx:
        fx=np.array(f(X[0],X[1]))
        fx=fx.squeeze()
        
        s=np.linalg.solve(jx,-fx)

        Xnew=X+s
        XnNorm=np.linalg.norm(Xnew,1)

        if XnNorm!=0:
            errorx=np.linalg.norm(s,1)/XnNorm
        else:
            errorx=np.linalg.norm(s,1)
        error.append(errorx)
        fnew=f(Xnew[0],Xnew[1])
        errorf=np.linalg.norm(fnew.squeeze(),1)

        X=Xnew
        it=it+1
    return X,it,error

#### Newton Rapshon-Shamaskii

In [11]:
def newton_rapshon_sham(init_guess,f,J,tolx,tolf,nmax):
    X=np.array(init_guess)
    
    it=0
    errorf=1+tolf
    errorx=1+tolx
    error=[]

    shamCounter=0
    while it<nmax and errorf>=tolf and errorx>=tolx:
        
        if shamCounter % 4 == 0: # Calculate jx only once every 4 steps
            jx=J(X[0],X[1])
        shamCounter=shamCounter + 1
        
        if np.linalg.det(jx)==0:
            print("J-Matrix does not have max rank")
            return None,None,None
        fx=np.array(f(X[0],X[1]))
        fx=fx.squeeze()
        
        s=np.linalg.solve(jx,-fx)

        Xnew=X+s
        XnNorm=np.linalg.norm(Xnew,1)

        if XnNorm!=0:
            errorx=np.linalg.norm(s,1)/XnNorm
        else:
            errorx=np.linalg.norm(s,1)
        error.append(errorx)
        fnew=f(Xnew[0],Xnew[1])
        errorf=np.linalg.norm(fnew.squeeze(),1)

        X=Xnew
        it=it+1
    return X,it,error

#### Minimum of Non Linear Function

In [12]:
def newton_rapshon_min(init_guess,grad_f_num,H_num,tolx,tolf,nmax):
    X=np.array(init_guess)
    
    it=0
    errorf=1+tolf
    errorx=1+tolx
    error=[]

    while it<nmax and errorf>=tolf and errorx>=tolx:
        Hx=H_num(X[0],X[1])
        if np.linalg.det(Hx)==0:
            print("H-Matrix does not have max rank")
            return None,None,None
        
        gfx=grad_f_num(X[0], X[1])
        gfx=gfx.squeeze()
        
        s=np.linalg.solve(Hx,-gfx)

        Xnew=X+s
        XnNorm=np.linalg.norm(Xnew,1)
        
        # Criteri di arresto
        if XnNorm!=0:
            errorx=np.linalg.norm(s,1)/XnNorm
        else:
            errorx=np.linalg.norm(s,1)

        gradf_xnew=grad_f_num(Xnew[0],Xnew[1])
        errorf=np.linalg.norm(gradf_xnew.squeeze(),1)
        
        error.append(errorx)

        X=Xnew
        it=it+1
    return X,it,error

### Linear Systems

#### Iterative Methods

##### Jacobi

In [13]:
def jacobi(A,b,x0,toll,it_max):
    errore=1000
    d=np.diag(A)
    n=A.shape[0]
    E=np.tril(A,-1)
    F=np.triu(A,1)
    N=-(E+F)

    # Convergencee Check (||T|| < 1 or sr < 1)
    T=np.dot(np.diag(1/d),N)
    eigenvalues=np.linalg.eigvals(T)
    spectralradius=np.max(np.abs(eigenvalues)) # sprad<1 -> Converge

    # print("Norm:",np.linalg.norm(T,2))
    
    it=0
    er_vet=[]
    while it<=it_max and errore>=toll:
        x=(b+N@x0)/d.reshape(n,1)
        errore=np.linalg.norm(x-x0)/np.linalg.norm(x)
        er_vet.append(errore)
        x0=x.copy()
        it=it+1
    return x,it,er_vet

##### Gauss Seidel

In [14]:
def gauss_seidel(A,b,x0,toll,it_max):
    errore=1000
    d=np.diag(A)
    D=np.diag(d)
    E=np.tril(A,-1)
    F=np.triu(A,1)
    M=D+E
    N=-F

    # Convergence check
    invM=np.linalg.inv(M)
    T=invM@N
    eigenvalues=np.linalg.eigvals(T)
    spectralradius=np.max(np.abs(eigenvalues))
    # print("raggio spettrale Gauss-Seidel ",raggiospettrale)

    # print("Norm:", np.linalg.norm(T,2))
    
    it=0
    er_vet=[]
    while it<=it_max and errore>=toll:
        x,flag=Lsolve(M,b-F@x0) #Calcolare la soluzione al passo k equivale a calcolare la soluzione del sistema triangolare con matrice M=D+E
                                # e termine noto b-F@x0
        errore=np.linalg.norm(x-x0)/np.linalg.norm(x)
        er_vet.append(errore)
        x0=x.copy()
        it=it+1
    return x,it,er_vet

##### Gauss Seidel SOR

In [15]:
def gauss_seidel_sor(A,b,x0,toll,it_max,omega):
    errore=1000
    d=np.diag(A)
    D=np.diag(d)
    Dinv=np.diag(1/d)
    E=np.tril(A,-1)
    F=np.triu(A,1)

    # Iteration Matrix
    Momega=D+omega*E
    Nomega=(1-omega)*D-omega*F
    T=np.dot(np.linalg.inv(Momega),Nomega)

    # Convergence Check
    eigenvalues=np.linalg.eigvals(T)
    spectralradius=np.max(np.abs(eigenvalues))
    # print("raggio spettrale Gauss-Seidel SOR ", raggiospettrale)
    
    M=D+E
    N=-F
    it=0
    xold=x0.copy()
    xnew=x0.copy()
    er_vet=[]
    while it<=it_max and errore>=toll:
        temp=b-np.dot(F,xold)
        xtilde,flag=Lsolve(M,temp)
        xnew=(1-omega)*xold+omega*xtilde
        errore=np.linalg.norm(xnew-xold)/np.linalg.norm(xnew)
        er_vet.append(errore)
        xold=xnew.copy()
        it=it+1
    return xnew,it,er_vet

#### Direct Methods

##### Gauss with Pivot

In [16]:
def LUsolve(A,b):
    PT,L,U=sp.linalg.lu(A)
    P=PT.T.copy()
    y,flag=st.Lsolve(L,P@b)
    if flag==0:
        x,flag=st.Usolve(U,y)
        if flag!=0:
            return [],flag
    else:
        return [],flag
    return x,flag

##### Householder Method ($QR$)

In [18]:
def QRsolve(A,b):
    Q,R=sp.linalg.qr(A)
    y=Q.T@b
    x,flag=st.Usolve(R,y)
    if flag != 0:
        return [], flag
    else:
        return x

#### Overdetermined Systems

##### Normal Equations

In [ ]:
def eqnorm(A,b):
    G=A.T@A
    
    print("Condition Number of G: ", np.linalg.cond(G))

    f=A.T@b
    
    if not is_symmetric(G) and not is_positively_defined(G):
        print('Requirements not Met')
        return None

    L=spLin.cholesky(G,lower=True)
    U=L.T

    z,flag=Lsolve(L,f)

    if flag==0:
        x,flag=Usolve(U,z)

    return x

##### QR Least Squares

In [ ]:
def qrLS(A,b):
    n=A.shape[1]
    Q,R=spLin.qr(A)
    
    h=Q.T@b
    
    x, flag =st.Usolve(R[:n,:n],h[:n])
    r=np.linalg.norm(h[n:])
    return x,r

##### Singular Value Decomposition Least Squares

In [ ]:
def SVDLS(A,b):
    m,n=A.shape  #numero di righe e  numero di colonne di A
    U,s,VT=spLin.svd(A)  
    
    V=VT.T
    thresh=np.spacing(1)*m*s[0] ##Calcolo del rango della matrice, numero dei valori singolari maggiori di una soglia
    k=np.count_nonzero(s>thresh)
    
    d=U.T@b
    d1=d[:k].reshape(k,1)
    s1=s[:k].reshape(k,1)
    
    c=d1/s1
    x=V[:,:k]@c 
    r=np.linalg.norm(d[k:])**2 
    return x,r

## Descent Methods
---

### Steepest Descent

In [ ]:
def steepestdescent(A,b,x0,itmax,tol):
    if not is_square(A):
        print('Matrix is not square')
        return [],[]
    
    x=x0.copy()
    r=A@x-b
    it=0
    normb=np.linalg.norm(b)

    error=np.linalg.norm(r)/normb

    vec_sol=[]
    vec_sol.append(x.copy())
    vec_r=[]
    vec_r.append(error)

    while errore>=tol and it<itmax:
        it=it+1
        
        Ap=A@p
        alpha=-(r.T@p)/(p.T@Ap)
        
        x=x+alpha*p
        r=r+alpha*Ap

        vec_sol.append(x.copy())
        error=np.linalg.norm(r)/normb
        vec_r.append(error)
        p=-r # Max descent (Opposite of gradient)
        
    iterates=np.vstack([arr.T for arr in vec_sol]) # Only for graphical purpose

    return x,vec_r, iterates, it

#### Conjugate Gradient

In [ ]:
def conjugateGradient(A,b,x0,itmax,tol):

    if not is_square(A):
        print('Matrix is not square')
        return [],[]
    
    x=x0.copy()
    r=A@x-b
    p=-r
    it=0
    errore=np.linalg.norm(r)/np.linalg.norm(b)
    
    vec_sol=[]
    vec_sol.append(x.copy())
    vec_r=[]
    vec_r.append(errore)

    while errore>=tol and it<itmax:
        it=it+1
        
        Ap=A@p

        save=(r.T@r)
        alpha=save/(Ap.T@p)
        
        x=x+alpha*p
        r=r+alpha*Ap

        vec_sol.append(x.copy())
        errore=np.linalg.norm(r)/normb
        vec_r.append(errore)

        gamma=(r.T@r)/save
        p=-r+gamma*p # Max descent (Opposite of gradient)
        
    iterates=np.vstack([arr.T for arr in vec_sol]) # Only for graphical purpose

    return x,vec_r, iterates, it

## Interpolation
---

### Lagrange Polynome

In [ ]:
def plagr(nodes,j):
    n=nodes.size
    zeros=np.zeros_like(nodes)
    
    zeros=np.append(nodes[:j],nodes[j+1:]) # one-line based programming.
    
    num=np.poly(zeros)
    den=np.polyval(num,nodes[j])

    p=num/den
    
    return p

### Interpolation With Lagrange Polynome

In [ ]:
def InterpL(x, y, xv):
     
     n=x.size
     m=xv.size
     L=np.zeros((m,n))
     for j in range(n):
        p=plagr(x,j)
        L[:,j]=np.polyval(p,xv)
         
     return L@y

### Cholesky Factorization

In [ ]:
def solve_cholesky(A,b):
    if is_positively_defined(A) and is_symmetric(A):
        L=cholesky(A,lower=True)
        y=np.linalg.solve(L,b)
        x=np.linal.solve(L.T,b)
        return x
    else:
        print('Matrix does not meet requirements')